## [RemoteCLIP🛰️: A Vision Language Foundation Model for Remote Sensing](https://arxiv.org/abs/2306.11029)

[Fan Liu (刘凡)](https://multimodality.group/author/%E5%88%98%E5%87%A1/)*
[Delong Chen (陈德龙)](https://chendelong.world/)*
Zhangqingyun Guan (管张青云)
Xiaocong Zhou (周晓聪)
Jiale Zhu (朱佳乐)
[Jun Zhou (周峻)](https://experts.griffith.edu.au/7205-jun-zhou)

\* Equal Contribution

[Github repository](https://github.com/ChenDelong1999/RemoteCLIP/)

[Huggingface model weights](https://huggingface.co/chendelong/RemoteCLIP/tree/main)

-> Run on GPU runtime (top right corner) <-

In [ ]:
# @title Install libraries
# %%capture
!pip install huggingface_hub open_clip_torch
!git clone https://github.com/ChenDelong1999/RemoteCLIP/

In [13]:
!pip install clip_benchmark

In [14]:
!pip install clip-benchmark

In [7]:
!pip install open_clip_torch

In [8]:
# @title Load packages and download model weights
from huggingface_hub import hf_hub_download
import torch, open_clip
from PIL import Image
from IPython.display import display

# for model_name in ['RN50'] #, 'ViT-B-32', 'ViT-L-14']: #faster loading
for model_name in ['RN50', 'ViT-B-32', 'ViT-L-14']: #all models
    checkpoint_path = hf_hub_download("chendelong/RemoteCLIP", f"RemoteCLIP-{model_name}.pt", cache_dir='checkpoints')
    print(f'{model_name} is downloaded to {checkpoint_path}.')

RemoteCLIP-RN50.pt:   0%|          | 0.00/408M [00:00<?, ?B/s]

RN50 is downloaded to checkpoints/models--chendelong--RemoteCLIP/snapshots/bf1d8a3ccf2ddbf7c875705e46373bfe542bce38/RemoteCLIP-RN50.pt.


RemoteCLIP-ViT-B-32.pt:   0%|          | 0.00/605M [00:00<?, ?B/s]

ViT-B-32 is downloaded to checkpoints/models--chendelong--RemoteCLIP/snapshots/bf1d8a3ccf2ddbf7c875705e46373bfe542bce38/RemoteCLIP-ViT-B-32.pt.


RemoteCLIP-ViT-L-14.pt:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

ViT-L-14 is downloaded to checkpoints/models--chendelong--RemoteCLIP/snapshots/bf1d8a3ccf2ddbf7c875705e46373bfe542bce38/RemoteCLIP-ViT-L-14.pt.


In [9]:
# @title Select Model
model_name = 'RN50' # @param ['RN50', 'ViT-B-32', 'ViT-L-14']
model, _, preprocess = open_clip.create_model_and_transforms(model_name)
tokenizer = open_clip.get_tokenizer(model_name)

path_to_your_checkpoints = 'checkpoints/models--chendelong--RemoteCLIP/snapshots/bf1d8a3ccf2ddbf7c875705e46373bfe542bce38'

ckpt = torch.load(f"{path_to_your_checkpoints}/RemoteCLIP-{model_name}.pt", map_location="cpu")
message = model.load_state_dict(ckpt)
print(message)
model = model.cuda().eval()

/tmp/ipykernel_31/3295391091.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(f"{path_to_your_checkpoints}/RemoteCLIP-{model_name}.pt", map_location="cp

<All keys matched successfully>


In [10]:
import os

# Updated paths for Kaggle
label_file_path = "/kaggle/input/cropped-dataset1/labels.txt"
true_labels = {}

# Read the labels
with open(label_file_path, "r") as f:
    for line in f:
        filename, label = line.strip().split("\t")
        true_labels[filename] = label

# Load image file names
image_folder = "/kaggle/input/cropped-dataset1/images"
image_files = [f for f in os.listdir(image_folder) if f.endswith(".jpg") or f.endswith(".png")]

print("Total images: ", len(image_files))

Total images:  38320


In [8]:
# import json

# label_file_path = "/kaggle/input/cropped-dataset1/labels.txt"
# image_dir = "/kaggle/input/cropped-dataset1/images"

# # Map each class to a descriptive caption
# class_to_caption = {
#     "vehicle": "A satellite view of various vehicles on a road or parking area.",
#     "baseballfield": "An aerial view of a baseball field with diamond shape.",
#     "windmill": "A large windmill or wind turbine seen from above.",
#     "bridge": "A bridge crossing over water or a road, viewed from space.",
#     "groundtrackfield": "A ground track and field stadium with running lanes.",
#     "overpass": "An overpass where one road crosses over another.",
#     "ship": "A large cargo ship or boat in a harbor or at sea.",
#     "airplane": "A commercial airplane on a runway or at an airport.",
#     "tenniscourt": "A tennis court with a net in the center seen from above.",
#     "Expressway-Service-area": "An expressway service area with parking and buildings.",
#     "basketballcourt": "A basketball court with hoops at both ends.",
#     "storagetank": "A circular or cylindrical storage tank from a top-down view.",
#     "chimney": "An industrial chimney or smokestack with emissions.",
#     "airport": "A busy airport with multiple runways and terminals.",
#     "stadium": "A stadium for sports or large events viewed from space.",
#     "Expressway-toll-station": "An expressway toll station with booths and vehicles.",
#     "dam": "A large dam holding back water, viewed from above.",
#     "golffield": "A golf course or golf field with green patches.",
#     "trainstation": "A train station with railway tracks and platforms.",
#     "harbor": "A harbor with many boats and loading docks.",
# }

# data = []
# with open(label_file_path, "r") as f:
#     for line in f:
#         filename, label = line.strip().split("\t")
#         caption = class_to_caption.get(label)
#         if caption:
#             data.append({
#                 "filename": filename,
#                 "split": "test",  # required by retrieval.py
#                 "sentences": [{"raw": caption}]
#             })

# json_output = {"images": data}
# with open("/kaggle/working/dataset_yours.json", "w") as out:
#     json.dump(json_output, out, indent=2)

# print("✅ JSON file written to /kaggle/working/dataset_yours.json")

✅ JSON file written to /kaggle/working/dataset_yours.json


In [21]:
# from tqdm import tqdm
# from torch.utils.data import Dataset, DataLoader
# from PIL import Image
# import argparse
# import os
# import random
# import torch
# import numpy as np
# import open_clip
# import json
# from clip_benchmark.metrics.zeroshot_retrieval import recall_at_k, batchify, dataloader_with_indices
# from clip_benchmark.datasets.builder import get_dataset_collate_fn
# import torch.nn.functional as F
# import gc

# def parse_args():
#     parser = argparse.ArgumentParser()
#     parser.add_argument("--model-name", type=str, choices=['RN50', 'ViT-B-32', 'ViT-L-14'])
#     parser.add_argument("--retrieval-json-dir", type=str, default=None)
#     parser.add_argument("--retrieval-images-dir", type=str, default="")
#     parser.add_argument("--remoteclip-path", type=str, default=None)
#     parser.add_argument("--batch-size", type=int, default=64)
#     parser.add_argument("--workers", type=int, default=8)
#     args, unknown = parser.parse_known_args()
#     if len(unknown) > 0:
#         print(f"[Unknown args]: {unknown}")
#     return args

# def get_model(args):
#     CLIP_model, preprocess_train, preprocess_val = open_clip.create_model_and_transforms(
#         model_name=args.model_name + '-quickgelu',
#         pretrained='openai',
#         device=args.device,
#         cache_dir='cache/weights/open_clip'
#     )
#     tokenize = open_clip.tokenize
#     checkpoint = torch.load(args.remoteclip_path, map_location="cuda")
#     msg = CLIP_model.load_state_dict(checkpoint)
#     print(msg)
#     return CLIP_model, preprocess_train, preprocess_val, preprocess_val, tokenize

# class JsonDataset(Dataset):
#     def __init__(self, json_dir, img_dir, transforms, max_samples=25000):
#         self.json_dir = json_dir
#         self.transforms = transforms
#         self.img_dir = img_dir
#         self.images = []
#         self.captions = []
#         self.read_json()
#         if max_samples and max_samples < len(self.images):
#             self.subsample(max_samples)
#         self.duplicate()

#     def read_json(self):
#         datasets = json.load(open(self.json_dir, "r"))
#         for image in datasets['images']:
#             if image['split'] == "test":
#                 for text in image['sentences']:
#                     self.images.append(image['filename'])
#                     self.captions.append(text['raw'].capitalize())

#     def subsample(self, max_samples):
#         indices = random.sample(range(len(self.images)), max_samples)
#         self.images = [self.images[i] for i in indices]
#         self.captions = [self.captions[i] for i in indices]

#     def duplicate(self):
#         unique_images, indexs = np.unique(self.images, return_index=True)
#         if len(unique_images) != len(self.images):
#             self.duplicated_images = []
#             self.duplicated_captions = []
#             for index in indexs:
#                 self.duplicated_images.append(self.images[index])
#                 same_indexs = [i for i, x in enumerate(self.images) if x == self.images[index]]
#                 captions = [self.captions[i] for i in same_indexs]
#                 self.duplicated_captions.append(captions)
#             self.images = self.duplicated_images
#             self.captions = self.duplicated_captions

#     def __len__(self):
#         return len(self.images)

#     def __getitem__(self, idx):
#         image_path = os.path.join(self.img_dir, self.images[idx])
#         image = Image.open(image_path).convert("RGB")
#         image = self.transforms(image)
#         text = self.captions[idx]
#         return image, text

# def retrieval_evaluation(args, model, preprocess, tokenize, recall_k_list=[1, 5, 10]):
#     print("🧩 Loading dataset with 25,000 random samples...")
#     dataset = JsonDataset(
#         args.retrieval_json_dir,
#         args.retrieval_images_dir,
#         preprocess,
#         max_samples=10000
#     )

#     dataloader = DataLoader(
#         dataset,
#         batch_size=args.batch_size,
#         num_workers=args.workers,
#         collate_fn=get_dataset_collate_fn('mscoco_captions')
#     )
#     n_batches = len(dataloader)

#     print("🔄 Extracting features...")
#     batch_images_emb_list = []
#     batch_texts_emb_list = []
#     texts_image_index = []
#     dataloader = dataloader_with_indices(dataloader)

#     for batch_images, batch_texts, inds in tqdm(dataloader, total=n_batches):
#         batch_images = batch_images.to(args.device)
#         batch_texts_image_index = [ind for ind, texts in zip(inds, batch_texts) for text in texts]
#         batch_texts = tokenize([text for texts in batch_texts for text in texts]).to(args.device)

#         with torch.no_grad():
#             image_features = model.encode_image(batch_images)
#             text_features = model.encode_text(batch_texts)
#             image_emb = F.normalize(image_features, dim=-1)
#             text_emb = F.normalize(text_features, dim=-1)

#         batch_images_emb_list.append(image_emb.cpu())
#         batch_texts_emb_list.append(text_emb.cpu())
#         texts_image_index.extend(batch_texts_image_index)

#     torch.cuda.empty_cache()
#     gc.collect()

#     print("🧠 Concatenating embeddings...")
#     images_emb = torch.cat(batch_images_emb_list)
#     texts_emb = torch.cat(batch_texts_emb_list)
#     batch_size = len(batch_images_emb_list[0])

#     del batch_images_emb_list, batch_texts_emb_list
#     torch.cuda.empty_cache()
#     gc.collect()

#     print("📊 Calculating similarity scores (chunked)...")
#     chunk_size = 5000
#     scores_list = []
#     for i in range(0, texts_emb.size(0), chunk_size):
#         with torch.no_grad():
#             chunk = texts_emb[i:i + chunk_size]
#             sim = chunk @ images_emb.T
#             scores_list.append(sim.cpu())
#     scores = torch.cat(scores_list, dim=0)

#     del chunk, sim, scores_list
#     torch.cuda.empty_cache()
#     gc.collect()

#     print("📈 Computing retrieval metrics...")
#     positive_pairs = torch.zeros_like(scores, dtype=bool)
#     positive_pairs[torch.arange(len(scores)), texts_image_index] = True

#     metrics = {}
#     for recall_k in recall_k_list:
#         metrics[f"retrieval-image2text-R@{recall_k}"] = (
#             batchify(recall_at_k, scores.T, positive_pairs.T, batch_size, args.device, k=recall_k) > 0
#         ).float().mean().item() * 100

#     for recall_k in recall_k_list:
#         metrics[f"retrieval-text2image-R@{recall_k}"] = (
#             batchify(recall_at_k, scores, positive_pairs, batch_size, args.device, k=recall_k) > 0
#         ).float().mean().item() * 100

#     metrics["retrieval-mean-recall"] = np.mean(list(metrics.values()))
#     for key in metrics:
#         metrics[key] = round(float(metrics[key]), 2)

#     print("🎯 Calculating classification accuracy...")
#     with torch.no_grad():
#         top1 = torch.topk(scores, 1, dim=1).indices.squeeze(1)
#         texts_image_index_tensor = torch.tensor(texts_image_index, dtype=torch.long)
#         correct_predictions = top1.cpu() == texts_image_index_tensor
#         metrics["classification-accuracy"] = round(correct_predictions.float().mean().item() * 100, 2)

#     print("📊 Calculating per-class accuracy...")
#     label_counts = {}
#     label_correct = {}
#     for text_idx, img_idx in enumerate(texts_image_index):
#         pred_img_idx = top1[text_idx].item()

#         true_caption = dataset.captions[img_idx][0] if isinstance(dataset.captions[img_idx], list) else dataset.captions[img_idx]
#         pred_caption = dataset.captions[pred_img_idx][0] if isinstance(dataset.captions[pred_img_idx], list) else dataset.captions[pred_img_idx]

#         label = true_caption.lower().strip()
#         pred = pred_caption.lower().strip()

#         label_counts[label] = label_counts.get(label, 0) + 1
#         if label == pred:
#             label_correct[label] = label_correct.get(label, 0) + 1

#     per_class_acc = {k: round(100 * label_correct.get(k, 0) / v, 2) for k, v in label_counts.items()}
#     metrics["per-class-accuracy"] = per_class_acc

#     # Final cleanup
#     del images_emb, texts_emb, scores, top1, correct_predictions, texts_image_index_tensor
#     torch.cuda.empty_cache()
#     gc.collect()

#     print("💾 Saving results...")
#     results_file = f"retrieval_results_{args.model_name}.json"
#     with open(results_file, "w") as f:
#         json.dump(metrics, f, indent=2)

#     print(f"✅ Results saved to {results_file}")
#     return metrics


In [20]:
# Delete any large known variables manually if they exist
for var in ['images_emb', 'texts_emb', 'scores', 'top1', 'correct_predictions', 
            'texts_image_index', 'label_counts', 'label_correct', 'dataset', 'model']:
    if var in globals():
        print(f"🔻 Deleting variable: {var}")
        del globals()[var]

# Run garbage collector and clear GPU memory
import gc
import torch

gc.collect()
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

print("✅ Explicitly cleared CPU and GPU memory.")

✅ Explicitly cleared CPU and GPU memory.


In [ ]:
# from types import SimpleNamespace
# import gc
# import torch

# model_list = ['RN50']
# retrieval_json = "/kaggle/working/dataset_yours.json"
# image_dir = "/kaggle/input/cropped-dataset1/images"
# results = {}

# for model_name in model_list:
#     print(f"\n🔧 Running evaluation for model: {model_name}")

#     remoteclip_path = f"checkpoints/models--chendelong--RemoteCLIP/snapshots/bf1d8a3ccf2ddbf7c875705e46373bfe542bce38/RemoteCLIP-{model_name}.pt"

#     args = SimpleNamespace(
#         model_name=model_name,
#         retrieval_json_dir=retrieval_json,
#         retrieval_images_dir=image_dir,
#         remoteclip_path=remoteclip_path,
#         batch_size=64,
#         workers=2,
#         device="cuda"
#     )

#     # Load model
#     model, preprocess_train, preprocess_val, preprocess_aug, tokenize = get_model(args)

#     # Run retrieval + accuracy + per-class
#     metrics = retrieval_evaluation(args, model, preprocess_aug, tokenize)
#     results[model_name] = metrics

#     # Aggressively free GPU and RAM memory before saving
#     del model, preprocess_train, preprocess_val, preprocess_aug, tokenize, args
#     torch.cuda.empty_cache()
#     gc.collect()



# print("\n📊 All model evaluations completed and saved.")


🔧 Running evaluation for model: RN50


/tmp/ipykernel_418/4104974055.py:37: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(args.remoteclip_path, map_location="cuda")


<All keys matched successfully>
🧩 Loading dataset with 25,000 random samples...
🔄 Extracting features...


100%|██████████| 157/157 [22:03<00:00,  8.43s/it]


🧠 Concatenating embeddings...
📊 Calculating similarity scores (chunked)...


In [ ]:
# model_list = ['ViT-B-32']
# retrieval_json = "/kaggle/working/dataset_yours.json"
# image_dir = "/kaggle/input/cropped-dataset1/images"
# results = {}

# for model_name in model_list:
#     print(f"\n🔧 Running evaluation for model: {model_name}")

#     remoteclip_path = f"checkpoints/models--chendelong--RemoteCLIP/snapshots/bf1d8a3ccf2ddbf7c875705e46373bfe542bce38/RemoteCLIP-{model_name}.pt"

#     args = SimpleNamespace(
#         model_name=model_name,
#         retrieval_json_dir=retrieval_json,
#         retrieval_images_dir=image_dir,
#         remoteclip_path=remoteclip_path,
#         batch_size=64,
#         workers=2,
#         device="cuda"
#     )

#     # Load model
#     model, preprocess_train, preprocess_val, preprocess_aug, tokenize = get_model(args)

#     # Run retrieval + accuracy + per-class
#     metrics = retrieval_evaluation(args, model, preprocess_aug, tokenize)
#     results[model_name] = metrics

#     # Aggressively free GPU and RAM memory before saving
#     del model, preprocess_train, preprocess_val, preprocess_aug, tokenize, args
#     torch.cuda.empty_cache()
#     gc.collect()



# print("\n📊 All model evaluations completed and saved.")